<a href="https://colab.research.google.com/github/sharanv16/AI-Game/blob/main/Copy_AI_Proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Forming the Grid

Create a closed square grid

*   1 implies closed cell
*   2 implies open cell
*   4 implies bot cell
*   8 implies captain cell
*   16 implies alien cell
*   32 implies bot getting caught by alien
*   64 implies alien occupying the captain cell


# SAVING PRIVATE CAPTAIN

## IMPORTS

In [ ]:
import inspect
import matplotlib.pyplot as plt
import numpy as np
import random
import time

## CONSTANTS

In [ ]:
BOT_FAILURE = "FAIL"
BOT_SUCCESS = "SUCCESS"
BOT_STUCK = "STUCK"

X_COORDINATE_SHIFT = [1, 0, 0, -1]
Y_COORDINATE_SHIFT = [0, 1, -1, 0]

LOG_NONE = 0
LOG_INFO = 1
LOG_DEBUG = 2

CLOSED_CELL = 1
OPEN_CELL = 2
BOT_CELL = 4
CAPTAIN_CELL = 8
ALIEN_CELL = 16
BOT_CAUGHT_CELL = 32
CAPTAIN_ALIEN_CELL = 64
BOT_SUCCESS_CELL = 128
MOVED_ALIEN_CELL = 256
MOVED_ALIEN_CAPTAIN_CELL = 512
ALL_ALIEN_CELLS = CAPTAIN_ALIEN_CELL | ALIEN_CELL
MOVEMENT_CELLS = OPEN_CELL | CAPTAIN_CELL
ALIEN_MOVEMENT_CELLS = MOVEMENT_CELLS | BOT_CELL
RESCUE_PATH = MOVEMENT_CELLS | CAPTAIN_ALIEN_CELL

NO_PATH_FOUND_FLAG = -1
FINDING_PATH_FLAG = 0
GOAL_REACHED_FLAG = 1
NO_MOVES_LEFT_FLAG = 2
BOT_CAUGHT_FLAG = 3

## GRID LOGIC

### GET NEIGHBORS

In [ ]:
"""
    Returns all adjacent cells that can be reached based on provided filter.

    Parameters
    ----------
    row_size - int
        size of the row : 10
    column_size - int
        size of the column : 10
    cell - list
        list containing x and y coordinates : (0, 0)
    filter - hex value
        filter cell based on hex value and only return those cells : 0xff
        0xff : adds all values
    grid - numpy matrix array
        grid to find adj cells details for filtering data

    Returns
    ----------
    set(list)
        Adjacent cell locations : [(1, 0), (0, 1), (2, 1), (1, 2)]
"""


def get_neighbors(row_size, column_size, cell, filter, grid):
    neighbors = []

    for i in range(4):
        x_cord = cell[0] + X_COORDINATE_SHIFT[i]
        y_cord = cell[1] + Y_COORDINATE_SHIFT[i]
        if (
            (0 <= x_cord < row_size)
            and (0 <= y_cord < column_size)
            and (grid[(x_cord, y_cord)] & filter)
        ):
            neighbors.append((x_cord, y_cord))

    return neighbors

In [ ]:
#Parameters grid size,number of aliens
class Ship:
  def __init__(self,n,no_of_aliens):
    self.no_of_aliens =no_of_aliens
    self.n = n
    self.grid = np.full((self.n,self.n),CLOSED_CELL)
    self.rows, self.cols = self.grid.shape
    # assign a random start point
    self.x_start =  random.randint(0,self.n-1)
    self.y_start =  random.randint(0,self.n-1)
    #print(f" start = {self.x_start} {self.y_start}")
    self.grid[self.x_start,self.y_start] = OPEN_CELL
    #start building paths from the start point

  def find_closed_cells_with_one_open_neighbor(self):
    self.results = []
    for i in range(self.rows):
      for j in range(self.cols):
        if self.grid[i, j] == -1:  # Check for closed cells only
          self.adjacent_cells = [(i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)]
          self.adjacent_cells = [(x, y) for x, y in self.adjacent_cells if 0 <= x < self.rows and 0 <= y < self.cols]
          self.num_zero_neighbors = sum(self.grid[x, y] == 0 for x, y in self.adjacent_cells)
          if self.num_zero_neighbors == 1:
            self.results.append((i, j))
    return self.results


  def create_grid(self):
    self.closed_cells = []
    self.avl_cells = self.find_closed_cells_with_one_open_neighbor()
    #print(self.avl_cells)
    while self.avl_cells:
      p = random.choice(self.avl_cells)
      self.grid[p] = OPEN_CELL
      self.avl_cells = []
      self.avl_cells = self.find_closed_cells_with_one_open_neighbor()
    for i in range(self.rows):
      for j in range(self.cols):
        if self.grid[i,j] ==-1:
          self.closed_cells.append((i,j))
    self.l = len(self.closed_cells)//2
    while self.l:
      random_cell = random.choice(self.closed_cells)
      self.grid[random_cell]  = OPEN_CELL
      self.closed_cells.remove(random_cell)
      self.l-=1



  def find_open_cells(self):
    self.open_cells = []
    for i in range(self.rows):
      for j in range(self.cols):
        if self.grid[i,j] == OPEN_CELL:
          self.open_cells.append((i,j))

  def place_aliens(self):
    k = self.no_of_aliens
    self.find_open_cells()
    self.alien_cells=[]
    while k:
      r = random.choice(self.open_cells)
      self.grid[r] = ALIEN_CELL
      self.alien_cells.append(r)
      self.open_cells.remove(r)
      k -=1


  def place_bot(self):
    r =  random.choice(self.open_cells)
    self.grid[r] = BOT_CELL
    self.start = r

  def place_captain(self):
    r = random.choice(self.open_cells)
    self.grid[r] = CAPTAIN_CELL
    self.goal = r

  def place_players(self):
    self.place_aliens()
    self.place_captain()
    self.place_bot()


  def show_grid(self):
    print(self.grid)



## VISUALIZING THE GRID

In [ ]:
def show_color_grid(grid):
    color_map = {
        CLOSED_CELL: "black",
        OPEN_CELL: "white",
        BOT_CELL: "blue",
        CAPTAIN_CELL: "green",
        ALIEN_CELL: "red",
        BOT_CAUGHT_CELL: "cyan",
        CAPTAIN_ALIEN_CELL: "magenta",
        BOT_SUCCESS_CELL: "yellow",
        MOVED_ALIEN_CELL: "red",
    }

    # Create the figure and axes
    fig, ax = plt.subplots()

    # Define cell width and height
    cell_width = 1
    cell_height = 1

    row_size, col_size = grid.shape

    # Fill the cells with the "x" pattern
    for i in range(row_size):
        for j in range(col_size):
            color = color_map[grid[i][j]]
            x = j * cell_width
            y = (row_size - i - 1) * cell_height
            ax.plot(
                [x, x + cell_width, x + cell_width, x],
                [y, y + cell_height, y, y + cell_height],
                color=color,
                linewidth=2,
            )

    # Draw black borders around each cell
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            ax.plot([j, j + 1], [i, i], color="black", linewidth=1)
            ax.plot([j, j + 1], [i + 1, i + 1], color="black", linewidth=1)
            ax.plot([j, j], [i, i + 1], color="black", linewidth=1)
            ax.plot([j + 1, j + 1], [i, i + 1], color="black", linewidth=1)

    # Set limits and labels
    ax.set_xlim(0, col_size * cell_width)
    ax.set_ylim(0, row_size * cell_height)
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    # Turn off axes
    ax.axis("off")

    # Show the plot
    plt.show()

## CONSTRAINTS

In [ ]:
class Constraints:
    def __init__(
        self,
        queue_size=-1,
        bot_max_moves=1000,
        max_search_time=-1,
        reduce_search_frequenzy=False,
    ):
        self.queue_size = queue_size
        self.bot_max_moves = bot_max_moves
        self.reduce_search_frequenzy = reduce_search_frequenzy
        self.max_search_time = max_search_time

## BOT LOGIC

### SEARCH ALGORITHM

In [ ]:
class Search_Algorithm:
    def __init__(self, ship, constraints, log_level):
        self.constraints = constraints
        self.local_ship = ship
        self.local_grid = np.copy(ship.grid)
        self.curr_pos = self.start_cell = ship.start
        self.captain_cell = ship.goal
        self.row_size = ship.row_size
        self.column_size = ship.column_size
        self.path_pos = 0
        self.log_level = log_level

    """
        Virtual Class that verifies if the neighbor was already visited.
        Additional conditions can be implemented in the child class.

        Parameters
        ----------
            neighbor - list
                list containing x and y cord : (0,0)
            visited_cells - set(list)
                set containing all the visited cells : [(0,0), (1,1), ...]

        Returns
        ----------
        boolean
            returns true if neighbor can be added to queue : true/false
    """

    def is_add_neighbor(self, grid_copy, neighbor, path_traversed):
        return neighbor not in self.visited_cells

    """
        Return the Shortest path from Bot cell to Captain cell

        Parameters
        ----------
            itr_completed - int
                no of moves completed by bot (movement + idle)
            grid_copy - numpy matrix array (optional)
                contains a copy of grid to find the shortest path, uses local copy of grid by default

        Returns
        ----------
        set(list)
            returns the shortest path found : [(0, 0), (0, 1), (1, 1), ... (x, y)]
    """

    def find_shortest_path(self, itr_count, neighbor_filter, grid_copy=None):
        bfs_queue = []
        self.visited_cells = set()
        self.path_pos = 0

        if grid_copy is None:
            grid_copy = self.local_grid

        # queue - [ (current_cell, (... path_traversed ...)), ... ]
        bfs_queue.append((self.curr_pos, [self.curr_pos]))

        while bfs_queue:
            current_cell, path_traversed = bfs_queue.pop(0)

            self.log_data(
                LOG_DEBUG,
                "current_cell",
                type(current_cell),
                len(current_cell),
                current_cell,
            )
            self.log_data(
                LOG_DEBUG,
                "path_traversed",
                type(path_traversed),
                len(path_traversed),
                path_traversed,
            )

            if current_cell == self.captain_cell:
                self.log_data(
                    LOG_DEBUG,
                    f"Iterations Completed : {itr_count}\tPath Length : {len(path_traversed)}\tBot Path : {path_traversed}",
                )
                return path_traversed
            elif current_cell in self.visited_cells:
                continue

            self.visited_cells.add(current_cell)
            self.log_data(
                LOG_DEBUG,
                "visited_cells",
                type(self.visited_cells),
                len(self.visited_cells),
                self.visited_cells,
            )

            current_cell_neighbors = get_neighbors(
                self.row_size,
                self.column_size,
                current_cell,
                neighbor_filter,
                grid_copy,
            )

            for neighbor in current_cell_neighbors:
                if (
                    self.constraints.queue_size == -1
                    or len(bfs_queue) <= self.constraints.queue_size
                ) and (self.is_add_neighbor(grid_copy, neighbor, path_traversed)):
                    bfs_queue.append((neighbor, path_traversed + [neighbor]))

        return None

    def sort_neighbors_by_alien(self, neighbor_cells, grid_copy):
        safest_neighbor = []
        for neighbor in neighbor_cells:
            adj_alien_cell_len = len(
                get_neighbors(
                    self.row_size,
                    self.column_size,
                    neighbor,
                    ALL_ALIEN_CELLS,
                    grid_copy,
                )
            )
            adj_alien_movements_len = len(
                get_neighbors(
                    self.row_size,
                    self.column_size,
                    neighbor,
                    MOVED_ALIEN_CELL | MOVED_ALIEN_CAPTAIN_CELL,
                    grid_copy,
                )
            )
            safest_neighbor.append(
                (neighbor, adj_alien_cell_len, adj_alien_movements_len)
            )

        safest_neighbor = sorted(
            sorted(safest_neighbor, key=lambda x: x[2]),
            key=lambda x: x[1],
        )

        return safest_neighbor

    def escape_to_nearest_open_cell(self, itr_count, grid_copy):
        neighbor_cells = get_neighbors(
            self.row_size,
            self.column_size,
            self.curr_pos,
            OPEN_CELL,
            grid_copy,
        )

        if len(neighbor_cells) == 0:
            return None

        safest_neighbor = self.sort_neighbors_by_alien(neighbor_cells, grid_copy)
        escape_path = [self.curr_pos, safest_neighbor[0][0]]
        self.log_data(
            LOG_DEBUG,
            f"Iterations Completed : {itr_count}\tEscape Path : {escape_path}",
        )

        return escape_path

    def escape_path_with_least_aliens(self, itr_count, grid_copy):
        adj_alien_movements = get_neighbors(
            self.row_size,
            self.column_size,
            self.curr_pos,
            MOVED_ALIEN_CAPTAIN_CELL,
            grid_copy,
        )

        if len(adj_alien_movements) > 0:  # handling edge cases, if any
            cap_path = [self.curr_pos, adj_alien_movements[0]]
            self.log_data(
                LOG_DEBUG,
                f"Iterations Completed : {itr_count}\tEscape Path : {cap_path}",
            )
            return cap_path

        adj_alien_movements = get_neighbors(
            self.row_size,
            self.column_size,
            self.curr_pos,
            MOVED_ALIEN_CELL,
            grid_copy,
        )
        if len(adj_alien_movements) == 0:
            return None

        safest_neighbor = self.sort_neighbors_by_alien(adj_alien_movements, grid_copy)
        escape_path = [self.curr_pos, safest_neighbor[0][0]]
        self.log_data(
            LOG_DEBUG,
            f"Iterations Completed : {itr_count}\tEscape Path : {escape_path}",
        )

        return escape_path

    def escape_nearby_aliens(self, itr_count, grid_copy):
        self.path_pos = 0

        if not (grid_copy[self.curr_pos] & BOT_CAUGHT_CELL):
            return None

        safe_path = self.escape_to_nearest_open_cell(itr_count, grid_copy)

        if safe_path is None:
            safe_path = self.escape_path_with_least_aliens(itr_count, grid_copy)

        return safe_path

### PARENT BOT

In [ ]:
class Parent_Bot(Search_Algorithm):
    def __init__(self, ship, constraints, log_level):
        super(Parent_Bot, self).__init__(ship, constraints, log_level)
        self.bot_path = [self.start_cell]
        self.alien_cells = ship.alien_cells.copy()
        self.visited_cells = self.bot_caught_cell = self.path = None
        self.idle_moves = self.bot_moves = 0
        self.status = BOT_FAILURE
        self.flag = FINDING_PATH_FLAG
        self.time_start = self.time_end = self.time_elapsed = 0

    def log_data(self, log_level, *args):
        if (self.log_level) and (log_level <= self.log_level):
            # print( self.__class__.__name__, "::", inspect.currentframe().f_back.f_code.co_name, "::", inspect.currentframe().f_back.f_lineno, "::", sep="", end="")
            print(*args)

    def display_grid(self, log_level):
        if (self.log_level) and (log_level <= self.log_level):
            self.log_data(LOG_INFO, self.local_grid)

    def start_timer(self):
        self.time_start = time.process_time_ns()

    def end_timer(self):
        self.time_end = time.process_time_ns()
        self.time_elapsed = self.time_end - self.time_start
        return self.time_elapsed

    def is_stop_search(self):
        if (self.constraints.max_search_time != -1) and (
            self.end_timer() > self.constraints.max_search_time
        ):
            return True

        self.bot_moves += 1
        if self.bot_moves > self.constraints.bot_max_moves:
            self.flag = NO_MOVES_LEFT_FLAG
            return True
        return False

    def move_bot(self):
        self.path_pos += 1
        next_cell = self.path[self.path_pos]
        retVal = False

        next_cell_val = self.local_grid[next_cell]
        if (next_cell_val & (ALIEN_CELL | CAPTAIN_ALIEN_CELL)):
            self.flag = BOT_CAUGHT_FLAG
            self.bot_caught_cell = next_cell
            self.local_grid[next_cell] = BOT_CAUGHT_CELL
            retVal = True
        elif next_cell_val & CAPTAIN_CELL:
            self.flag = GOAL_REACHED_FLAG
            self.local_grid[next_cell] = BOT_SUCCESS_CELL
            retVal = True
        else:
            self.local_grid[next_cell] = BOT_CELL

        self.local_grid[self.curr_pos] = OPEN_CELL
        self.curr_pos = next_cell
        self.bot_path.append(next_cell)

        return retVal

    def move_aliens(self):
        random.shuffle(self.alien_cells)

        for itr, alien in enumerate(self.alien_cells):
            alien_moves_possible = get_neighbors(
                self.row_size,
                self.column_size,
                alien,
                ALIEN_MOVEMENT_CELLS,
                self.local_grid,
            )
            self.log_data(
                LOG_DEBUG, f"Alien {itr, alien} has moves {alien_moves_possible}"
            )

            if len(alien_moves_possible) == 0:
                self.log_data(
                    LOG_DEBUG,
                    f"Alien {itr, alien} has no moves",
                )
                continue

            alien_new_cell = random.choice(alien_moves_possible)

            if self.local_grid[alien_new_cell] & BOT_CELL:
                self.log_data(
                    LOG_DEBUG,
                    f"Alien moves from current cell {itr, alien} to bot cell {alien_new_cell}",
                )
                self.flag = BOT_CAUGHT_FLAG
                self.bot_caught_cell = alien_new_cell
                self.local_grid[alien_new_cell] = BOT_CAUGHT_CELL
                self.local_grid[alien] = OPEN_CELL
                return True
            elif self.local_grid[alien_new_cell] & CAPTAIN_CELL:
                self.log_data(
                    LOG_DEBUG,
                    f"Alien moves from current cell {itr, alien} to captain cell {alien_new_cell}",
                )
                self.local_grid[alien_new_cell] = CAPTAIN_ALIEN_CELL
                self.local_grid[alien] = OPEN_CELL
            else:
                if self.local_grid[alien] & CAPTAIN_ALIEN_CELL:
                    self.log_data(
                        LOG_DEBUG,
                        f"Alien moves from captain cell {itr, alien, self.local_grid[alien]} to open cell {alien_new_cell}",
                    )
                    self.local_grid[alien] = CAPTAIN_CELL
                else:
                    self.log_data(
                        LOG_DEBUG,
                        f"Alien moves from current cell {itr, alien, self.local_grid[alien]} to open cell {alien_new_cell}",
                    )
                    self.local_grid[alien] = OPEN_CELL
                self.local_grid[alien_new_cell] = ALIEN_CELL

            # Update alien_cells with new location
            self.alien_cells[itr] = alien_new_cell

        return False

    def print_rescue_status(self):
        if self.flag == GOAL_REACHED_FLAG:
            self.log_data(LOG_INFO, "Goal Reached!!")
            self.status = BOT_SUCCESS
        elif self.flag == NO_MOVES_LEFT_FLAG:
            self.log_data(LOG_INFO, "Bot is stuck, no moves left!!")
            self.display_grid(LOG_INFO)
            self.status = BOT_STUCK
        elif self.flag == BOT_CAUGHT_FLAG:
            self.log_data(LOG_INFO, "RIP Bot!!")
            self.log_data(LOG_INFO, f"Alien caught bot at {self.bot_caught_cell}")
        elif self.flag == NO_PATH_FOUND_FLAG:
            self.log_data(LOG_INFO, "No path found!!")
        else:
            self.log_data(LOG_INFO, "Unknown error occurred!!")

    def print_rescue_output(self):
        self.end_timer()
        self.print_rescue_status()
        self.log_data(LOG_INFO, self.bot_path)
        self.log_data(LOG_INFO, f"no of steps taken = {len(self.bot_path)}")
        if self.idle_moves:
            self.log_data(LOG_INFO, f"no of idle moves = {self.idle_moves}")
        self.log_data(LOG_INFO, f"total iterations = {self.bot_moves}")

### BOT 1

In [ ]:
class Bot_1(Parent_Bot):
    def __init__(self, ship, constraints, log_level=LOG_INFO):
        super(Bot_1, self).__init__(ship, constraints, log_level)

    def start_rescue(self):
        self.start_timer()
        self.path = self.find_shortest_path(self.bot_moves, RESCUE_PATH | ALIEN_CELL)

        if self.path is None:
            self.flag = NO_PATH_FOUND_FLAG
        else:
            while self.flag == FINDING_PATH_FLAG:
                if self.is_stop_search():
                    break

                self.display_grid(LOG_DEBUG)
                if self.move_bot():
                    break

                if self.move_aliens():
                    self.display_grid(LOG_DEBUG)
                    break

        self.print_rescue_output()

### BOT 2

In [ ]:
class Bot_2(Parent_Bot):
    def __init__(self, ship, constraints, log_level=LOG_INFO):
        super(Bot_2, self).__init__(ship, constraints, log_level)

    def is_recalculate_path(self):
        return (self.path is None) or (self.path in self.alien_cells)

    def start_rescue(self):
        self.start_timer()
        while self.flag == FINDING_PATH_FLAG:
            if self.is_stop_search():
                break

            if (
                not self.constraints.reduce_search_frequenzy
                or self.is_recalculate_path()
            ):
                self.path = self.find_shortest_path(self.bot_moves, MOVEMENT_CELLS)

            self.display_grid(LOG_DEBUG)
            if self.path is None:
                self.idle_moves += 1
            elif self.move_bot():
                break

            if self.move_aliens():
                self.display_grid(LOG_DEBUG)
                break

        self.print_rescue_output()

### BOT 3

In [ ]:
class Bot_3(Bot_2):
    def __init__(self, ship, constraints, log_level=LOG_INFO):
        super(Bot_3, self).__init__(ship, constraints, log_level)

    def clone_grid_with_alien_moves(self):
        clone_grid = np.copy(self.local_grid)

        for alien in self.alien_cells:
            adj_cells = get_neighbors(
                self.row_size, self.column_size, alien, MOVEMENT_CELLS, self.local_grid
            )
            for cell in adj_cells:
                clone_grid[cell] = (
                    MOVED_ALIEN_CELL
                    if clone_grid[cell] & OPEN_CELL
                    else MOVED_ALIEN_CAPTAIN_CELL
                )

        return clone_grid

    def start_rescue(self):
        self.start_timer()
        while self.flag == FINDING_PATH_FLAG:
            if self.is_stop_search():
                break

            if (
                not self.constraints.reduce_search_frequenzy
            ) or self.is_recalculate_path():
                grid_alien_move_copy = self.clone_grid_with_alien_moves()
                self.path = self.find_shortest_path(
                    self.bot_moves, MOVEMENT_CELLS, grid_alien_move_copy
                )
                if self.path is None:
                    self.path = self.find_shortest_path(self.bot_moves, MOVEMENT_CELLS)
                    if self.path is None:
                        self.idle_moves += 1

            self.display_grid(LOG_DEBUG)
            if (self.path is not None) and (self.move_bot()):
                break

            if self.move_aliens():
                self.display_grid(LOG_DEBUG)
                break

        self.print_rescue_output()

### BOT 4

In [ ]:
class Bot_4(Bot_3):
    def __init__(self, ship, constraints, log_level=LOG_INFO):
        super(Bot_4, self).__init__(ship, constraints, log_level)
        self.is_escape_path = False

    def clone_grid_with_alien_moves(self):
        clone_grid = np.copy(self.local_grid)

        for alien in self.alien_cells:
            adj_cells = get_neighbors(
                self.row_size,
                self.column_size,
                alien,
                MOVEMENT_CELLS | BOT_CELL,
                self.local_grid,
            )
            for cell in adj_cells:
                if clone_grid[cell] & OPEN_CELL:
                    clone_grid[cell] = MOVED_ALIEN_CELL
                elif clone_grid[cell] & CAPTAIN_CELL:
                    clone_grid[cell] = MOVED_ALIEN_CAPTAIN_CELL
                else:
                    clone_grid[cell] = BOT_CAUGHT_CELL

        return clone_grid

    def is_add_neighbor(self, grid_copy, neighbor, path_traversed):
        ## BOT 4 V2 Logic
        if len(path_traversed) < 6 and grid_copy[neighbor] & (
            ALL_ALIEN_CELLS | MOVED_ALIEN_CELL | MOVED_ALIEN_CAPTAIN_CELL
        ):
            return False

        return super(Bot_4, self).is_add_neighbor(grid_copy, neighbor, path_traversed)

    def is_recalculate_path(self):
        if self.path is None:
            return True

        is_cap_next_pos = self.path_pos + 1
        if (is_cap_next_pos > len(self.path)) and (
            self.local_grid[self.path[is_cap_next_pos]] & CAPTAIN_CELL
        ):
            return False

        for itr in range(1, 5, 1):
            next_pos = self.path_pos + 1
            if next_pos < len(self.path):
                if self.path[next_pos] in self.alien_cells:
                    return True

                adj_cells_for_next_move = get_neighbors(
                    self.row_size,
                    self.column_size,
                    self.path[next_pos],
                    ALL_ALIEN_CELLS,
                    self.local_grid,
                )
                if adj_cells_for_next_move in self.alien_cells:
                    return True

        return False

    def start_rescue(self):
        self.start_timer()
        while self.flag == FINDING_PATH_FLAG:
            if self.is_stop_search():
                break

            if (
                (not self.constraints.reduce_search_frequenzy)
                or self.is_escape_path
                or self.is_recalculate_path()
            ):
                self.is_escape_path = False
                grid_alien_move_copy = self.clone_grid_with_alien_moves()

                ## BOT 4 V1 Logic
                # self.path = self.find_shortest_path(
                #     self.bot_moves, MOVEMENT_CELLS, grid_alien_move_copy
                # )

                ## BOT 4 V2 Logic
                self.path = self.find_shortest_path(
                    self.bot_moves, RESCUE_PATH | MOVED_ALIEN_CAPTAIN_CELL | MOVED_ALIEN_CELL | ALIEN_CELL, grid_alien_move_copy
                )
                if self.path is None:
                    self.path = self.escape_nearby_aliens(
                        self.bot_moves, grid_alien_move_copy
                    )
                    if self.path is None:
                        self.idle_moves += 1  # Sit and pray
                    else:
                        self.is_escape_path = True

            self.display_grid(LOG_DEBUG)

            if (self.path is not None) and (self.move_bot()):
                break

            if self.move_aliens():
                self.display_grid(LOG_DEBUG)
                break

        self.print_rescue_output()

# MAIN

In [ ]:
ship = Ship(30, 10)
ship.create_grid()
ship.place_players()
# constraints = Constraints()
constraints = Constraints(
    1000,
    int(ship.grid.shape[0] * ship.grid.shape[1] * 0.8),
    -1,
    True,
)

In [ ]:
ship.show_grid()
show_color_grid(ship.grid)
print(f"bot position {ship.start}\tcaptain position {ship.goal}")
print(
    f"distance = {np.sqrt((ship.start[0] - ship.goal[0])**2 + (ship.start[1]-ship.goal[1])**2)}"
)

In [ ]:
bot1 = Bot_1(ship, constraints)
bot1.start_rescue()

In [ ]:
bot2 = Bot_2(ship, constraints)
bot2.start_rescue()

In [ ]:
bot3 = Bot_3(ship, constraints)
bot3.start_rescue()

In [ ]:
bot4 = Bot_4(ship, constraints)
bot4.start_rescue()

In [ ]:
del bot4
del bot3
del bot2
del bot1
del constraints
del ship

# PLOTTING THE DATA

In [ ]:
import csv
import os
import sys

# Create a list of column headings
column_headings = [
    "BOT NAME",
    "GRID SIZE",
    "START POSITION",
    "GOAL POSITION",
    "LAST BOT POSITION",
    "NUMBER OF ALIENS",
    "CONSTRAINT LEVEL",
    "MAX BOT BUFFER SIZE",
    "MAX MOVES ALLOWED",
    "GOAL DISTANCE",
    "STATUS",
    "PATH TRAVERSED",
    "IDLE MOVES",
    "ITERATIONS",
    "SEARCH TIME"
]


def constraints_factory(constraints_level, ship):
    open_cells = 1000
    max_moves_allowed = ship.grid.shape[0] * ship.grid.shape[1]
    if constraints_level == 0:
        return Constraints(-1, max_moves_allowed)  # No constraints,
    if constraints_level == 1:
        return Constraints(
            int(open_cells * 0.8), int(max_moves_allowed * 0.8), -1, True
        )  # max queue size is capped at 80% the number of available cells
    elif constraints_level == 2:
        return Constraints(
            int(open_cells * 2 / 3), int(max_moves_allowed * 0.8), -1, True
        )  # capped at 66.6667%
    elif constraints_level == 3:
        return Constraints(
            int(open_cells * 0.5), int(max_moves_allowed * 0.8), -1, True
        )  # capped to 50%


def bot_factory(itr, ship, constraints):
    bot_no = itr % 4
    if bot_no == 0:
        return Bot_1(ship, constraints, LOG_NONE)
    elif bot_no == 1:
        return Bot_2(ship, constraints, LOG_NONE)
    elif bot_no == 2:
        return Bot_3(ship, constraints, LOG_NONE)
    elif bot_no == 3:
        return Bot_4(ship, constraints, LOG_NONE)


def run_simulations(itr, ship, goal_distance, writer):
    try:
        constraints_level = int(itr / 4)
        constraints = constraints_factory(constraints_level, ship)
        bot = bot_factory(itr, ship, constraints)
        bot.start_rescue()

        # with new logic, this needs to be updated!!!
        length_of_path = bot.bot_moves - bot.idle_moves

        writer.writerow(
            [
                bot.__class__.__name__,  # name of bot
                f"{ship.grid.shape[0]} x { ship.grid.shape[1]}",  # size of grid
                ship.start,  # bot starting cell
                ship.goal,  # captain cell
                bot.curr_pos,  # last known position of bot
                len(ship.alien_cells),  # number of aliens
                constraints_level,  # level of constrain set
                constraints.queue_size,
                constraints.bot_max_moves,
                goal_distance,  # length from start to goal, hypotonuse
                bot.status,  # bot result
                length_of_path,  # bot path length
                bot.idle_moves,  # number of idle moves by bot
                bot.bot_moves,  # total number of moves made by bot
                bot.time_elapsed,
            ]
        )

        del bot
        del constraints
    except Exception as e:
        print(f"Error occurred during iteration {itr}.\n", e, file=sys.stderr)
        sys.exit(1)


print(f"CSV file generation started")
timer_start = time.process_time()

# Open a new CSV file in write mode
with open("alien_battle_simulation.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(column_headings)
    for i in range(0, 100):
        ship = Ship(30, 30)
        ship.generate_grid()
        ship.place_players()

        goal_distance = np.sqrt(
            (ship.start[0] - ship.goal[0]) ** 2 + (ship.start[1] - ship.goal[1]) ** 2
        )

        print('Roomba is trying to save for the %d\'th time.' % i, end='\r')
        for itr in range(16):
            run_simulations(itr, ship, goal_distance, writer)


timer_end = time.process_time()
timer_elapsed = timer_end - timer_start
print(f"CSV file created successfully!\n Time taken{timer_elapsed}")